In [1]:
import numpy as np
from numpy import genfromtxt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, f_classif,  SelectPercentile
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from random import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import SelectKBest
import pandas as pd

In [34]:
path = '/home/nicobruno/flor/Tesis/'
sujeto = '12_auditivo'

matriz = path + 'resultados/'+'S'+ sujeto+'_evoked_all_marker.csv'
vector = path + 'vectores/'+'S_'+sujeto+'.csv'

lista=2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,5,26,27,28,29,30


eeg= genfromtxt(matriz, skip_header=1,usecols=(lista),delimiter=',')
labels= genfromtxt(vector, skip_header=1,delimiter=',')

In [35]:

nfolds = 10
kf = KFold(n_splits=nfolds) # validación cruzada (número de folds)


all_auc = []
all_auc_sh = []
niter = 100
k = 5
selected = np.zeros((niter*nfolds,29))


n=-1
for i in np.arange(niter):
    print(i)
    for train, test in kf.split(eeg):
        n=n+1
        temp = []
        temp_sh= []
        X_train, X_test, y_train, y_test = eeg[train], eeg[test], labels[train], labels[test]
        # ABAJO ESTÁ LA GRILLA DE SELECCION DE PARAMETROS. PROBAR CON DISTINTOS RANGOS Y PASOS ALPHA
        
        
        selector = SelectKBest(k=k)
        selector.fit(X_train, y_train)
        X_train_new = selector.transform(X_train)
        X_test_new = selector.transform(X_test)
        selected[n,:] = selector.get_support()

        clf = RandomForestClassifier(n_estimators=1000)
        clf.fit(X_train_new,y_train)
        grid_predictions_stack = clf.predict_proba(X_test_new)
        temp.append(roc_auc_score(y_test, grid_predictions_stack[:,1]))
                    
        y_train_sh = y_train.copy()  
        shuffle(y_train_sh)

        clf.fit(X_train_new,y_train_sh)
        grid_predictions_stack = clf.predict_proba(X_test_new)
        temp_sh.append(roc_auc_score(y_test, grid_predictions_stack[:,1]))
                       
    all_auc.append(np.mean(temp))
    all_auc_sh.append(np.mean(temp_sh))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [36]:
prueba = np.sum(all_auc_sh>all_auc)/100
p = np.sum(np.array(all_auc_sh)>np.array(all_auc))/100

print(prueba)
print(p)

0.0
0.13


In [33]:
auc_1 = '/home/nicobruno/flor/Tesis/gradient_boosting_selector_resultados/S09_visual_auc_gb.csv'
auc_sh_1 = '/home/nicobruno/flor/Tesis/gradient_boosting_selector_resultados/S09_visual_auc_sh_gb.csv'

auc_datos= genfromtxt(auc_1, skip_header=1,delimiter=',')
auc_sh_datos= genfromtxt(auc_sh_1, skip_header=1,delimiter=',')

p = np.sum(np.array(auc_sh_datos)>np.array(auc_datos))/100
print(p)

0.01


In [37]:
folder = '/home/nicobruno/flor/Tesis/gradient_boosting_selector_resultados/'

In [38]:
pd.DataFrame(data={
    "values": all_auc
}).to_csv(folder+'S'+sujeto + '_' + 'auc_gb.csv', index=False)

In [39]:
pd.DataFrame(data={
    "values": all_auc_sh
}).to_csv(folder+'S'+sujeto + '_' + 'auc_sh_gb.csv', index=False)